# Convert to YOLOv8 Supported Dataset
Taking dataset provided here https://github.com/yastrebksv/TennisCourtDetector and translating it to Ultralytics supported Yolov8 pose detection models.

The original dataset has no bounding box. So I just made the bounding box center at the center of the image and width and height to be the size of the image. The original json describing one image was as such.

```json
[
    {
        "id": "PuXlxKdUIes_2450", 
        "metric": 0.28256459215943674, 
        "kps": [[378, 186], [905, 184], [242, 566], [1058, 564], [445, 185], [345, 566], [839, 185], [956, 564], [428, 247], [858, 246], [377, 445], [919, 443], [643, 247], [648, 444]]
        }
]
```

Since there was no visible/visible but blocked/invisible. I treated all keypoint markers (kpos) as visible as long as the normalized coordinatest fit within the image (values between 0 and 1). Keypoints outside this range were set to have values 0.000000, 0.000000, 0.00000. This is done to ensure that examples are not marked by corrupted. However, for this dataset, it seems that most points are visible.


In [1]:
import os
import json
from PIL import Image
import matplotlib.pyplot as plt
import shutil

# Converting Train and Validate
Below are the scripts to convert to correct YOLOv8 dataset folder hiearchy as well as label format.

In [2]:
orig_path = "data"
orig_images = os.listdir(orig_path + "/images")
data_train = "data/data_train.json"

print(len(orig_images))

# Make sure all images are same width and height
widths = []
heights = []
for img in orig_images:
    width, height = Image.open(orig_path + "/images/" + img).size
    widths.append(width)
    heights.append(height)
    
print("Widths: ", widths)
print("Heights: ", heights)

image_height = 720
image_width = 1280

center_x = image_width / 2
center_y = image_height / 2


8841
Widths:  [1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1280, 1

In [3]:

# Define the paths
yolo_data_path = "yolo_data"
train_images_path = os.path.join(yolo_data_path, "images", "train")
train_labels_path = os.path.join(yolo_data_path, "labels", "train")

# Create the directories if they don't exist
os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)

# Read the data_train.json file
with open(data_train, "r") as f:
    data = json.load(f)

# Loop through the data and process each image
for item in data:
    image_name = item["id"]
    kps = item["kps"]
    
    # Copy the image to the train images folder
    original_image_path = os.path.join(orig_path, "images", image_name + ".png")
    new_image_path = os.path.join(train_images_path, image_name + ".png")

    # Example usage
    shutil.copyfile(original_image_path, new_image_path)
    
    # Create the label file
    label_file_path = os.path.join(train_labels_path, image_name + ".txt")
    
    with open(label_file_path, "w") as label_file:
        # Write the class name (0) as the first line
        label_file.write("0 ")
        
        # Write the bounding box coordinates
        label_file.write(f"{center_x / image_width:.6f} {center_y / image_height:.6f} 1.000000 1.000000 ")
        
        # Write the keypoints coordinates
        for kp in kps:
            x, y = kp
            # Normalize
            x = x / image_width
            y = y / image_height
            v = 2.0

            # If either of the coordinates are out of bounds [0, 1], fill with 0
            if x < 0 or x > 1 or y < 0 or y > 1:
                x = 0
                y = 0
                v = 0
    
            label_file.write(f"{x:.6f} {y:.6f} {v:.6f} ")

In [4]:
import shutil
# Define the paths
yolo_data_path = "yolo_data"
valid_images_path = os.path.join(yolo_data_path, "images", "val")
valid_labels_path = os.path.join(yolo_data_path, "labels", "val")

# Create the directories if they don't exist
os.makedirs(valid_images_path, exist_ok=True)
os.makedirs(valid_labels_path, exist_ok=True)

# Read the data_valid.json file
data_valid = "data/data_val.json"
with open(data_valid, "r") as f:
    data = json.load(f)

# Loop through the data and process each image
for item in data:
    image_name = item["id"]
    kps = item["kps"]
    
    # Copy the image to the valid images folder
    original_image_path = os.path.join(orig_path, "images", image_name + ".png")
    new_image_path = os.path.join(valid_images_path, image_name + ".png")

    # Example usage
    shutil.copyfile(original_image_path, new_image_path)
    
    # Create the label file
    label_file_path = os.path.join(valid_labels_path, image_name + ".txt")
    
    with open(label_file_path, "w") as label_file:
        # Write the class name (0) as the first line
        label_file.write("0 ")
        
        # Write the bounding box coordinates
        label_file.write(f"{center_x / image_width:.6f} {center_y / image_height:.6f} 1.000000 1.000000 ")
        
        # Write the keypoints coordinates
        for kp in kps:
            # Normalize
            x = x / image_width
            y = y / image_height
            v = 2.0

            # If either of the coordinates are out of bounds [0, 1], fill with 0
            if x < 0 or x > 1 or y < 0 or y > 1:
                x = 0
                y = 0
                v = 0
    
            label_file.write(f"{x:.6f} {y:.6f} {v:.6f} ")